# Building Footprint Extraction for China

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/building_footprints_china.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [2]:
pip install geoai-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.0/281.0 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.9/632.9 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.5/122.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.8/241.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/

## Import libraries

In [3]:
import geoai

## Download sample data

In [17]:
raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/buildings_china.tif"
)

In [18]:
raster_path = geoai.download_file(raster_url)

## Initialize the model

In [8]:
extractor = geoai.BuildingFootprintExtractor(model_path="building_footprints_usa.pth")

Model path not specified, downloading from Hugging Face...


building_footprints_usa.pth:   0%|          | 0.00/176M [00:00<?, ?B/s]

Model downloaded to: /root/.cache/huggingface/hub/models--giswqs--geoai/snapshots/089548329c81f128fa12576663e7abdedb5cfa0e/building_footprints_usa.pth
Model loaded successfully


## Extract building footprints

In [19]:
masks_path = extractor.generate_masks(
    raster_path,
    output_dir="building_masks.tif",
    min_object_area=1000,
    confidence_threshold=0.5,
    mask_threshold=0.6,
    threshold=0.5,
)

Dataset initialized with 6 rows and 10 columns of chips
Image dimensions: 3654 x 2083 pixels
Chip size: 512 x 512 pixels
Overlap: 25.0% (stride_x=384, stride_y=384)
CRS: EPSG:3857
Processing raster with 15 batches


100%|██████████| 15/15 [08:21<00:00, 33.41s/it]

Masks with confidence values saved to buildings_china_masks_conf.tif


In [20]:
geoai.view_raster(masks_path, opacity=0.7, colormap="tab20", basemap=raster_path)

## Vectorize masks

In [21]:
gdf = geoai.orthogonalize(
    input_path=masks_path, output_path="building_footprints.geojson", epsilon=1.0
)

Processing 278 features...


Converting features: 100%|██████████| 278/278 [00:04<00:00, 60.72shape/s]


Saving to building_footprints.geojson...
Done!


## Add geometric attributes

In [22]:
gdf = geoai.add_geometric_properties(gdf)

## Visualize results

In [23]:
geoai.view_vector_interactive(
    gdf, style_kwds={"color": "red", "fillOpacity": 0.4}, tiles=raster_url
)

![image](https://github.com/user-attachments/assets/de8d0328-17c2-4858-a255-efc4a51dae40)